In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010030rest 20160324 1054..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020025rest 20150713 1519..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010013rest 20150703 1333..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020016rest 20150701 1040..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020015_rest 20150630 1527.csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010022restnew 20150724 14.csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02020027rest 20150713 1049..csv
/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2/02010008_rest 20150619 1653.csv
/kaggle/input/preprocessed-raw-m

# **Setup, load & preprocessing, save splits**

In [ ]:
# CELL 1: Setup + Load + Preprocess + Save splits
import os, re, math, json, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import IncrementalPCA

# CONFIG
DATA_DIR    = '/kaggle/input/preprocessed-raw-mat-csv/mat-csv-actual/mat-csv-actual/csv_files_from_mat2'
OUTPUT_DIR  = '/kaggle/working/dl_results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

SAMPLE_FRAC      = 1.0        # set 0.1 for quick tests
USE_IPCA         = True
IPCA_COMPONENTS  = 128
IPCA_BATCH       = 5000
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# GPU memory growth (optional)
gpus = tf.config.experimental.list_physical_devices('GPU')
for g in gpus:
    try:
        tf.config.experimental.set_memory_growth(g, True)
    except Exception:
        pass

# helpers
def detect_eeg_columns(columns):
    regex = re.compile(r'^(?:EEG[_\-\s]?|E[_\-\s]?)(0*?)(\d{1,3})$', flags=re.I)
    found = {}
    for c in columns:
        m = regex.match(c.strip())
        if m:
            num = int(m.group(2))
            if 1 <= num <= 128:
                found[num] = c
    if found:
        return [found[i] for i in sorted(found.keys())]
    # fallback
    return [c for c in columns if re.match(r'^(E|EEG)\d+', c, flags=re.I)]

def to_binary_label_series(s):
    s = s.dropna()
    if s.empty: return None
    s_num = pd.to_numeric(s, errors='coerce')
    if s_num.notna().all():
        uniq = set(np.unique(s_num))
        if uniq.issubset({0,1}): return s_num.astype(int)
        if uniq.issubset({1,2}): return s_num.map({1:0,2:1}).astype(int)
        med = float(s_num.median()); return (s_num > med).astype(int)
    s_str = s.astype(str)
    unique_vals = s_str.unique()
    if len(unique_vals) == 1: return s_str.map({unique_vals[0]:0}).astype(int)
    if len(unique_vals) == 2:
        le = LabelEncoder().fit(unique_vals)
        return pd.Series(le.transform(s_str), index=s_str.index).astype(int)
    mode_val = s_str.mode().iat[0]; return (s_str != mode_val).astype(int)

# 1) Read CSVs
csvs = sorted([f for f in os.listdir(DATA_DIR) if f.endswith('.csv')])
if len(csvs)==0:
    raise RuntimeError("No CSV files in DATA_DIR")
print("Found", len(csvs), "CSV files.")

parts = []
for fn in csvs:
    path = os.path.join(DATA_DIR, fn)
    df = pd.read_csv(path, engine='python')
    if SAMPLE_FRAC is not None and 0 < SAMPLE_FRAC < 1.0:
        df = df.sample(frac=SAMPLE_FRAC, random_state=SEED)
    df['__source_file'] = os.path.splitext(fn)[0]
    parts.append(df)
combined = pd.concat(parts, ignore_index=True)
print("Combined shape:", combined.shape)

# 2) label detection (prefer epoch, label, condition)
label_cols_try = ['epoch','label','condition','cond','target']
label_series = None
for c in label_cols_try:
    if c in combined.columns:
        s = to_binary_label_series(combined[c])
        if s is not None:
            label_series = pd.Series(index=combined.index, dtype=int)
            label_series.loc[combined[c].dropna().index] = s
            label_series = label_series.fillna(0).astype(int)
            print("Using", c, "as labels.")
            break
if label_series is None:
    # fallback search
    for c in combined.columns:
        if c.startswith('__'): continue
        s = to_binary_label_series(combined[c])
        if s is not None:
            label_series = pd.Series(index=combined.index, dtype=int)
            label_series.loc[combined[c].dropna().index] = s
            label_series = label_series.fillna(0).astype(int)
            print("Fallback using", c, "as labels.")
            break
if label_series is None:
    raise RuntimeError("No suitable label column found. Ensure 'epoch'/'label' exists.")

print("Label distribution:", label_series.value_counts().to_dict())
if label_series.nunique() <= 1:
    print("Detected single class after mapping — abort and inspect label columns.")
    raise RuntimeError("Single-class dataset. Fix labels.")

combined['__label'] = label_series.astype(int)

# 3) Detect EEG columns & form feature matrix
eeg_cols = detect_eeg_columns(combined.columns)
if not eeg_cols:
    raise RuntimeError("No EEG columns detected; check column names.")
print("Detected EEG columns:", len(eeg_cols))
# drop known metadata columns
drop_cols = {'time','condition','label','epoch','__source_file','__label'}
feature_cols = [c for c in eeg_cols if c not in drop_cols]
if len(feature_cols) == 0:
    raise RuntimeError("No feature columns after filtering.")
X_full = combined[feature_cols].to_numpy(dtype=np.float32)
y = combined['__label'].to_numpy(dtype=np.int32)
print("X_full shape:", X_full.shape, "y shape:", y.shape)

# impute NaNs
if np.isnan(X_full).any():
    col_means = np.nanmean(X_full, axis=0)
    inds = np.where(np.isnan(X_full)); X_full[inds] = np.take(col_means, inds[1])
    print("Imputed NaNs.")

# 4) Optional IncrementalPCA
if USE_IPCA and IPCA_COMPONENTS is not None and 0 < IPCA_COMPONENTS < X_full.shape[1]:
    print("Running IncrementalPCA...")
    ipca = IncrementalPCA(n_components=IPCA_COMPONENTS)
    n = X_full.shape[0]; bs = IPCA_BATCH
    for i in range(0, n, bs):
        ipca.partial_fit(X_full[i:i+bs])
    X_reduced = np.empty((n, IPCA_COMPONENTS), dtype=np.float32)
    for i in range(0, n, bs):
        X_reduced[i:i+bs] = ipca.transform(X_full[i:i+bs]).astype(np.float32)
    X = X_reduced
else:
    X = X_full
print("Post-PCA shape:", X.shape)

# 5) scale and split (save splits for model cells)
scaler = StandardScaler()
X = scaler.fit_transform(X).astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# persist splits so model cells can load them
np.savez_compressed(os.path.join(OUTPUT_DIR, 'data_split.npz'),
                    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
print("Saved data_split.npz to", OUTPUT_DIR)
# create empty models_results.json if not exists
res_path = os.path.join(OUTPUT_DIR, 'models_results.json')
if not os.path.exists(res_path):
    with open(res_path,'w') as f: json.dump([], f)
print("Cell 1 done.")


# **Utility functions**

In [3]:
# CELL 2: Utility functions for model cells (run once)
import os, json, numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score, accuracy_score

OUTPUT_DIR = '/kaggle/working/dl_results'
def load_data_splits():
    p = os.path.join(OUTPUT_DIR, 'data_split.npz')
    d = np.load(p)
    return d['X_train'], d['X_test'], d['y_train'], d['y_test']

def save_model_result(res):
    """Append JSON-serializable result dict to models_results.json"""
    p = os.path.join(OUTPUT_DIR, 'models_results.json')
    lst = []
    if os.path.exists(p):
        with open(p,'r') as f:
            try:
                lst = json.load(f)
            except Exception:
                lst = []
    lst.append(res)
    with open(p,'w') as f:
        json.dump(lst, f)

def make_result_dict(name, model, X_test, y_test, history=None):
    # predict probabilities where possible
    try:
        probs = model.predict(X_test, verbose=0).ravel()
    except Exception:
        # if model expects 3D or 4D, let caller reshape X_test appropriately before calling make_result_dict
        probs = model.predict(X_test, verbose=0).ravel()
    preds = (probs >= 0.5).astype(int)
    acc = float(accuracy_score(y_test, preds))
    try:
        roc_auc = float(roc_auc_score(y_test, probs))
    except Exception:
        roc_auc = None
    rep = classification_report(y_test, preds)
    cm = confusion_matrix(y_test, preds).tolist()
    try:
        fpr,tpr,_ = roc_curve(y_test, probs)
        fpr = fpr.tolist(); tpr = tpr.tolist()
    except Exception:
        fpr,tpr = [], []
    hist_dict = history.history if history is not None else {}
    # convert numpy types in hist to lists
    clean_hist = {k: (list(np.array(v).astype(float)) if hasattr(v,'__iter__') else v) for k,v in hist_dict.items()}
    res = {
        'name': name,
        'accuracy': acc,
        'roc_auc': roc_auc,
        'class_report': rep,
        'conf_mat': cm,
        'fpr': fpr,
        'tpr': tpr,
        'history': clean_hist
    }
    return res

print("Cell 2 loaded utilities.")

Cell 2 loaded utilities.


# **GAN**

# **GAN-Fixed**

In [4]:
# ================================
# CELL 3 — Vanilla GAN (1000 epochs) - Kaggle-optimized
# Saves BEST + LAST weights for Generator & Discriminator
# ================================
import os, time, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

# ---------------- Config ----------------
OUTPUT_DIR = "/kaggle/working/dl_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

LATENT_DIM = 100
EPOCHS_GAN = 1000
BATCH = 64
CLASSIFIER_EPOCHS = 50
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ---------------- Load data ----------------
X_train, X_test, y_train, y_test = load_data_splits()
X = X_train.astype(np.float32)
FEATURES = X.shape[1]
N_train = X.shape[0]

print(f"[GAN] features={FEATURES} | n_train={N_train}")

# ---------------- Build models ----------------
def build_generator():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(256, input_dim=LATENT_DIM),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(1024),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(FEATURES, activation='tanh')
    ])

def build_discriminator():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(512, input_shape=(FEATURES,)),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(256),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

generator = build_generator()
discriminator = build_discriminator()

opt_d = tf.keras.optimizers.Adam(0.0002, 0.5)
opt_g = tf.keras.optimizers.Adam(0.0002, 0.5)

# ---------------- GAN model ----------------
z = tf.keras.Input(shape=(LATENT_DIM,))
fake_x = generator(z)
discriminator.trainable = False
gan = tf.keras.Model(z, discriminator(fake_x))
gan.compile(optimizer=opt_g, loss="binary_crossentropy")
discriminator.compile(optimizer=opt_d, loss="binary_crossentropy")

# ---------------- tf.data pipeline ----------------
AUTOTUNE = tf.data.AUTOTUNE
ds = tf.data.Dataset.from_tensor_slices(X).shuffle(10000).repeat().batch(BATCH).prefetch(AUTOTUNE)
ds_iter = iter(ds)

real_label = tf.ones((BATCH,1), tf.float32)
fake_label = tf.zeros((BATCH,1), tf.float32)

# ---------------- Training step ----------------
@tf.function
def train_step(real_batch):
    # --- Train Discriminator on real & fake ---
    noise = tf.random.normal((BATCH, LATENT_DIM))
    fake_batch = generator(noise, training=True)

    with tf.GradientTape() as t1:
        pred_real = discriminator(real_batch, training=True)
        loss_real = tf.reduce_mean(tf.keras.losses.binary_crossentropy(real_label, pred_real))

    grads_real = t1.gradient(loss_real, discriminator.trainable_variables)
    opt_d.apply_gradients(zip(grads_real, discriminator.trainable_variables))

    with tf.GradientTape() as t2:
        pred_fake = discriminator(fake_batch, training=True)
        loss_fake = tf.reduce_mean(tf.keras.losses.binary_crossentropy(fake_label, pred_fake))

    grads_fake = t2.gradient(loss_fake, discriminator.trainable_variables)
    opt_d.apply_gradients(zip(grads_fake, discriminator.trainable_variables))

    # --- Train Generator ---
    noise2 = tf.random.normal((BATCH, LATENT_DIM))
    with tf.GradientTape() as t3:
        gen_out = generator(noise2, training=True)
        disc_out = discriminator(gen_out, training=False)
        loss_g = tf.reduce_mean(tf.keras.losses.binary_crossentropy(real_label, disc_out))

    grads_g = t3.gradient(loss_g, generator.trainable_variables)
    opt_g.apply_gradients(zip(grads_g, generator.trainable_variables))

    return (loss_real + loss_fake) * 0.5, loss_g

# ---------------- Adversarial Training ----------------
print("[GAN] Training started...")
d_losses, g_losses = [], []
best_g_loss = 999999.0
gen_best_path = os.path.join(OUTPUT_DIR, "generator_best_main.weights.h5")

start = time.time()
for epoch in range(EPOCHS_GAN):
    real_batch = next(ds_iter)
    d_loss_val, g_loss_val = train_step(real_batch)

    d_losses.append(float(d_loss_val))
    g_losses.append(float(g_loss_val))

    # Save BEST generator
    if epoch == 0 or float(g_loss_val) < best_g_loss:
        best_g_loss = float(g_loss_val)
        generator.save_weights(gen_best_path)

    if epoch % 50 == 0:
        print(f"[{epoch}/{EPOCHS_GAN}] D={d_losses[-1]:.4f} | G={g_losses[-1]:.4f}")

print(f"[GAN] Finished in {time.time()-start:.1f}s")

print(f"BEST generator loss = {best_g_loss:.5f}")
print(f"Saved BEST generator weights → {gen_best_path}")

# ---------------- Classifier Training ----------------
def mixed_batch_generator(X_real, y_real, batch):
    n = X_real.shape[0]
    half = batch // 2
    while True:
        idx = np.random.randint(0, n, half)
        real_x = X_real[idx]
        real_y = y_real[idx].reshape(-1,1).astype(np.float32)

        noise = np.random.normal(0,1,(half,LATENT_DIM)).astype(np.float32)
        gen_x = generator.predict(noise, verbose=0)
        gen_y = np.zeros((half,1), np.float32)

        Xb = np.vstack([real_x, gen_x])
        yb = np.vstack([real_y, gen_y])
        perm = np.random.permutation(len(Xb))

        yield Xb[perm], yb[perm]

train_gen = mixed_batch_generator(X, y_train, BATCH)
steps_per_epoch = max(10, N_train // BATCH)

disc_best_path = os.path.join(OUTPUT_DIR, "disc_classifier_best_main.weights.h5")

checkpoint = ModelCheckpoint(disc_best_path, monitor="val_loss",
                             save_best_only=True, save_weights_only=True)
early = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

discriminator.trainable = True
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002,0.5),
                      loss="binary_crossentropy", metrics=["accuracy"])

history = discriminator.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=CLASSIFIER_EPOCHS,
    validation_data=(X_test, y_test.reshape(-1,1)),
    callbacks=[checkpoint, early],
    verbose=1
)

# ---------------- Evaluation ----------------
y_prob = discriminator.predict(X_test).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

fpr, tpr, _ = roc_curve(y_test, y_prob)
auc_val = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"AUC={auc_val:.3f}")
plt.plot([0,1],[0,1],'k--')
plt.legend()
plt.title("GAN Discriminator ROC")
plt.savefig(os.path.join(OUTPUT_DIR,"gan_discriminator_roc.png"), dpi=100)
plt.close()

plt.figure(figsize=(6,3))
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.savefig(os.path.join(OUTPUT_DIR,"gan_disc_acc.png"), dpi=100)
plt.close()

plt.figure(figsize=(6,3))
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.savefig(os.path.join(OUTPUT_DIR,"gan_disc_loss.png"), dpi=100)
plt.close()

# ---------------- Save JSON ----------------
res = make_result_dict("Vanilla_GAN_best_gen_and_disc", discriminator, X_test, y_test, history)
res.update({
    "best_generator_weights": os.path.basename(gen_best_path),
    "best_discriminator_weights": os.path.basename(disc_best_path),
})
save_model_result(res)

# ---------------- SAVE LAST EPOCH WEIGHTS ----------------
gen_last_path = os.path.join(OUTPUT_DIR, "generator_last_main.weights.h5")
disc_last_path = os.path.join(OUTPUT_DIR, "discriminator_last_main.weights.h5")

generator.save_weights(gen_last_path)
discriminator.save_weights(disc_last_path)

print(f"[SAVED] Last generator weights → {gen_last_path}")
print(f"[SAVED] Last discriminator weights → {disc_last_path}")

# ---------------- Cleanup ----------------
K.clear_session()
gc.collect()

print("🔥 CELL 3 DONE — Best + Last weights saved for Generator & Discriminator.")


[GAN] features=128 | n_train=3089910


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1763837289.140756      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763837289.141483      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


[GAN] Training started...


ValueError: in user code:

    File "/tmp/ipykernel_48/1379056478.py", line 90, in train_step  *
        opt_d.apply_gradients(zip(grads_real, discriminator.trainable_variables))
    File "/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py", line 382, in apply_gradients  **
        grads, trainable_variables = zip(*grads_and_vars)

    ValueError: not enough values to unpack (expected 2, got 0)


In [ ]:
# ================================
# CELL 3 — Vanilla GAN (1000 epochs) - Kaggle-optimized
# Saves BEST + LAST weights for Generator & Discriminator
# ================================
import os, time, gc
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

# ---------- Config ----------
OUTPUT_DIR = "/kaggle/working/dl_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)

LATENT_DIM = 100
EPOCHS_GAN = 1000
BATCH = 64
CLASSIFIER_EPOCHS = 50
SEED = 42
np.random.seed(SEED); tf.random.set_seed(SEED)

# ---------- Load pre-saved splits (Cell 1/2 required) ----------
X_train, X_test, y_train, y_test = load_data_splits()
X = X_train.astype(np.float32)
FEATURES = X.shape[1]
N_train = X.shape[0]
print(f"[GAN] features={FEATURES} | n_train={N_train}")

# ---------- Build Generator & Discriminator (unchanged architecture) ----------
def build_generator():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(256, input_dim=LATENT_DIM),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(512),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(1024),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dense(FEATURES, activation='tanh')
    ], name="generator")

def build_discriminator():
    return tf.keras.Sequential([
        tf.keras.layers.Dense(512, input_shape=(FEATURES,)),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(256),
        tf.keras.layers.LeakyReLU(0.2),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ], name="discriminator")

generator = build_generator()
discriminator = build_discriminator()

opt_d = tf.keras.optimizers.Adam(0.0002, 0.5)
opt_g = tf.keras.optimizers.Adam(0.0002, 0.5)

# ---------- Build GAN wrapper (for generator training via BCE) ----------
z = tf.keras.Input(shape=(LATENT_DIM,))
fake_x = generator(z)
# freeze discriminator when building gan model (standard)
discriminator.trainable = False
gan = tf.keras.Model(z, discriminator(fake_x))
gan.compile(optimizer=opt_g, loss="binary_crossentropy")
# compile discriminator standalone (we'll use custom training steps)
discriminator.compile(optimizer=opt_d, loss="binary_crossentropy")

# ---------- tf.data pipeline for streaming real batches ----------
AUTOTUNE = tf.data.AUTOTUNE
ds = tf.data.Dataset.from_tensor_slices(X).shuffle(10000, seed=SEED).repeat().batch(BATCH).prefetch(AUTOTUNE)
ds_iter = iter(ds)

real_label = tf.ones((BATCH,1), tf.float32)
fake_label = tf.zeros((BATCH,1), tf.float32)

# ---------- Ensure trainable flags are correct before custom training ----------
discriminator.trainable = True
generator.trainable = True

# ---------- Robust @tf.function train step (handles None grads) ----------
@tf.function
def train_step(real_batch):
    # Train Discriminator on real examples
    noise = tf.random.normal((BATCH, LATENT_DIM))
    fake_batch = generator(noise, training=True)

    with tf.GradientTape() as t1:
        pred_real = discriminator(real_batch, training=True)
        loss_real = tf.reduce_mean(tf.keras.losses.binary_crossentropy(real_label, pred_real))
    grads_real = t1.gradient(loss_real, discriminator.trainable_variables)
    grads_and_vars = [(g, v) for g, v in zip(grads_real, discriminator.trainable_variables) if g is not None]
    if grads_and_vars:
        opt_d.apply_gradients(grads_and_vars)

    # Train Discriminator on fake examples
    with tf.GradientTape() as t2:
        pred_fake = discriminator(fake_batch, training=True)
        loss_fake = tf.reduce_mean(tf.keras.losses.binary_crossentropy(fake_label, pred_fake))
    grads_fake = t2.gradient(loss_fake, discriminator.trainable_variables)
    grads_and_vars = [(g, v) for g, v in zip(grads_fake, discriminator.trainable_variables) if g is not None]
    if grads_and_vars:
        opt_d.apply_gradients(grads_and_vars)

    # Train Generator (via GAN objective: make discriminator predict real)
    noise2 = tf.random.normal((BATCH, LATENT_DIM))
    with tf.GradientTape() as t3:
        gen_out = generator(noise2, training=True)
        # evaluate discriminator in inference mode
        disc_out = discriminator(gen_out, training=False)
        loss_g = tf.reduce_mean(tf.keras.losses.binary_crossentropy(real_label, disc_out))
    grads_g = t3.gradient(loss_g, generator.trainable_variables)
    grads_and_vars = [(g, v) for g, v in zip(grads_g, generator.trainable_variables) if g is not None]
    if grads_and_vars:
        opt_g.apply_gradients(grads_and_vars)

    return (loss_real + loss_fake) * 0.5, loss_g

# ---------- Adversarial training loop (memory-safe streaming) ----------
print("[GAN] Adversarial training started...")
d_losses, g_losses = [], []
best_g_loss = np.inf
gen_best_path = os.path.join(OUTPUT_DIR, "generator_best_main.weights.h5")

start_time = time.time()
for epoch in range(EPOCHS_GAN):
    real_batch = next(ds_iter)  # streaming minibatch
    d_loss_val, g_loss_val = train_step(real_batch)

    d_losses.append(float(d_loss_val))
    g_losses.append(float(g_loss_val))

    # Save BEST generator weights (tiny) when g_loss improves
    if float(g_loss_val) < best_g_loss:
        best_g_loss = float(g_loss_val)
        try:
            generator.save_weights(gen_best_path)
        except Exception as e:
            print("Warning saving best generator weights:", e)

    if epoch % 50 == 0:
        print(f"[Epoch {epoch}/{EPOCHS_GAN}] D={d_losses[-1]:.4f} | G={g_losses[-1]:.4f}")

print(f"[GAN] Adversarial training finished in {(time.time()-start_time):.1f}s")
print(f"[GAN] Best generator loss: {best_g_loss:.6f} → saved to {gen_best_path}")

# ---------- Train discriminator as classifier using streamed synthetic (no large saves) ----------
def mixed_batch_generator(X_real, y_real, batch):
    n = X_real.shape[0]
    half = batch // 2
    while True:
        idx = np.random.randint(0, n, half)
        real_x = X_real[idx]
        real_y = y_real[idx].reshape(-1,1).astype(np.float32)

        noise = np.random.normal(0,1,(half, LATENT_DIM)).astype(np.float32)
        gen_x = generator.predict(noise, verbose=0)
        gen_y = np.zeros((half,1), dtype=np.float32)

        Xb = np.vstack([real_x, gen_x])
        yb = np.vstack([real_y, gen_y])
        perm = np.random.permutation(len(Xb))

        yield Xb[perm], yb[perm]

train_gen = mixed_batch_generator(X, y_train, BATCH)
steps_per_epoch = max(10, N_train // BATCH)

disc_best_path = os.path.join(OUTPUT_DIR, "disc_classifier_best_main.weights.h5")
# ModelCheckpoint requires .weights.h5 when save_weights_only=True
checkpoint = ModelCheckpoint(disc_best_path, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=0)
early = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

# Re-compile discriminator for classifier fine-tuning
discriminator.trainable = True
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5),
                      loss="binary_crossentropy", metrics=["accuracy"])

# Use compact validation set (X_test) for quick val checks
val_data = (X_test.astype(np.float32), y_test.reshape(-1,1).astype(np.float32))

print("[Classifier] Fine-tuning discriminator as classifier (streamed synthetic)...")
history = discriminator.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=CLASSIFIER_EPOCHS,
    validation_data=val_data,
    callbacks=[checkpoint, early],
    verbose=1
)

# ---------- Save LAST weights (tiny) ----------
gen_last_path = os.path.join(OUTPUT_DIR, "generator_last_main.weights.h5")
disc_last_path = os.path.join(OUTPUT_DIR, "discriminator_last_main.weights.h5")
try:
    generator.save_weights(gen_last_path)
    discriminator.save_weights(disc_last_path)
except Exception as e:
    print("Warning saving last weights:", e)

# ---------- Evaluate on X_test and save small PNGs & JSON ----------
y_prob = discriminator.predict(X_test.astype(np.float32), verbose=0).ravel()
y_pred = (y_prob >= 0.5).astype(int)

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

# ROC + AUC
try:
    auc_val = float(roc_auc_score(y_test, y_prob))
except Exception:
    auc_val = None

fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, label=f"AUC={auc_val:.3f}" if auc_val is not None else "ROC")
plt.plot([0,1],[0,1],'k--', alpha=0.3)
plt.title("GAN Discriminator ROC")
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.legend(loc="lower right")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,"gan_discriminator_roc.png"), dpi=100)
plt.close()

# Train/Val curves (tiny)
plt.figure(figsize=(6,3))
if 'accuracy' in history.history:
    plt.plot(history.history.get('accuracy', []), label='train_acc')
    plt.plot(history.history.get('val_accuracy', []), label='val_acc')
    plt.legend(); plt.title("Train/Val Accuracy")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR,"gan_disc_acc.png"), dpi=100)
    plt.close()

plt.figure(figsize=(6,3))
plt.plot(history.history.get('loss', []), label='train_loss')
plt.plot(history.history.get('val_loss', []), label='val_loss')
plt.legend(); plt.title("Train/Val Loss")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,"gan_disc_loss.png"), dpi=100)
plt.close()

# Save small JSON summary via helper
res = make_result_dict("Vanilla_GAN_best_gen_and_disc", discriminator, X_test, y_test, history)
res.update({
    "best_generator_weights": os.path.basename(gen_best_path),
    "best_discriminator_weights": os.path.basename(disc_best_path) if os.path.exists(disc_best_path) else None,
    "last_generator_weights": os.path.basename(gen_last_path),
    "last_discriminator_weights": os.path.basename(disc_last_path),
    "gan_epochs": int(EPOCHS_GAN)
})
save_model_result(res)
print("[JSON] GAN result appended to models_results.json")

# ---------- Cleanup ----------
K.clear_session()
del train_gen, generator, discriminator, gan, ds_iter
gc.collect()

print("CELL 3 DONE — Best + Last weights saved for Generator & Discriminator.")
print("Files in:", OUTPUT_DIR)

[GAN] features=128 | n_train=3089910
[GAN] Adversarial training started...
[Epoch 0/1000] D=0.7015 | G=0.7190
[Epoch 50/1000] D=0.5996 | G=1.1754
[Epoch 100/1000] D=0.5972 | G=0.9374
[Epoch 150/1000] D=0.5452 | G=0.8271
[Epoch 200/1000] D=0.7037 | G=0.8502
[Epoch 250/1000] D=0.5765 | G=1.0324
[Epoch 300/1000] D=0.7250 | G=0.9082
[Epoch 350/1000] D=0.5633 | G=1.0090
[Epoch 400/1000] D=0.7418 | G=0.8414
[Epoch 450/1000] D=0.7056 | G=1.1800
[Epoch 500/1000] D=0.6641 | G=0.8385
[Epoch 550/1000] D=0.6552 | G=0.8614
[Epoch 600/1000] D=0.6244 | G=1.0132
[Epoch 650/1000] D=0.6242 | G=0.9307
[Epoch 700/1000] D=0.6283 | G=0.8464
[Epoch 750/1000] D=0.6960 | G=0.7569
[Epoch 800/1000] D=0.6430 | G=0.8459
[Epoch 850/1000] D=0.7464 | G=0.7527
[Epoch 900/1000] D=0.7422 | G=0.7379
[Epoch 950/1000] D=0.7057 | G=0.8253
[GAN] Adversarial training finished in 9.9s
[GAN] Best generator loss: 0.502903 → saved to /kaggle/working/dl_results/generator_best_main.weights.h5
[Classifier] Fine-tuning discriminator 

I0000 00:00:1763837653.680750     114 service.cc:148] XLA service 0x79fb24013700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1763837653.681592     114 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1763837653.681611     114 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1763837653.751627     114 cuda_dnn.cc:529] Loaded cuDNN version 90300


Epoch 1/50


I0000 00:00:1763837654.046785     114 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


48279/48279 ━━━━━━━━━━━━━━━━━━━━ 3066s 63ms/step - accuracy: 0.7800 - loss: 0.3409 - val_accuracy: 0.6469 - val_loss: 0.5980
Epoch 2/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2807s 58ms/step - accuracy: 0.8178 - loss: 0.3043 - val_accuracy: 0.6775 - val_loss: 0.5630
Epoch 3/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2823s 58ms/step - accuracy: 0.8288 - loss: 0.2919 - val_accuracy: 0.7003 - val_loss: 0.5385
Epoch 4/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2819s 58ms/step - accuracy: 0.8358 - loss: 0.2845 - val_accuracy: 0.7114 - val_loss: 0.5238
Epoch 5/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2802s 58ms/step - accuracy: 0.8408 - loss: 0.2785 - val_accuracy: 0.7201 - val_loss: 0.5129
Epoch 6/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2818s 58ms/step - accuracy: 0.8439 - loss: 0.2752 - val_accuracy: 0.7304 - val_loss: 0.5004
Epoch 7/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2820s 58ms/step - accuracy: 0.8468 - loss: 0.2715 - val_accuracy: 0.7398 - val_loss: 0.4907
Epoch 8/50
48279/48279 ━━━━━━━━━━━━━━━━━━━━ 2803s 58ms/step